In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
cnbv_df = pd.read_csv( "base_cnbv.csv")
catalogo_sector= pd.read_csv('catalogo.csv', encoding= 'latin-1')

In [3]:
cnbv_df["Cartera_Total"] = cnbv_df["Cartera_Total"].str.replace(",","").astype(float)

In [4]:
cnbv_df['Cartera_Total'].sum()

2564433.3899999997

In [5]:
catalogo_sector.head()

,Actividad_economica_agragada,Sector
0,"Agricultura, Silvicultura, Ganadería y Pesca",Otros
1,"Alimentos, Bebidas y Tabaco",Otros
2,Comercio,Otros
3,Comunicaciones y Telecomunicaciones,Otros
4,Construcción,Construccion


In [6]:
catalogo_sector.head()

,Actividad_economica_agragada,Sector
0,"Agricultura, Silvicultura, Ganadería y Pesca",Otros
1,"Alimentos, Bebidas y Tabaco",Otros
2,Comercio,Otros
3,Comunicaciones y Telecomunicaciones,Otros
4,Construcción,Construccion


In [7]:
merge_1 = pd.merge(cnbv_df, catalogo_sector, on='Actividad_economica_agragada')

In [8]:
merge_1['Sector'].unique()

array(['Otros', 'Construccion', 'Industrias manufactureras',
       'Mineria y energia', 'Actividades secundarias'], dtype=object)

In [9]:
merge_1.columns

Index(['Escala', 'Estado_calculado', 'Actividad_economica_agragada',
       'Destino_del_crédito', 'Institución', 'Periodo_seguimiento', 'Cartera',
       'Cartera_Total', 'Cartera_Vigente', 'Cartera_Vencida', 'IMOR',
       'Creditos', 'Acreditados', 'Sector'],
      dtype='object')

In [10]:
rds_connection_string =  "postgres:root@localhost:5432/CARTERA_VS_PIB"

In [11]:
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
PIB_df=pd.read_sql_query('select * from pib_data', con=engine)

In [13]:
PIB_df.head()

,Estado_calculado,Actividad_economica_agragada,2018/01,2018/02,2018/03,2018/04,2018/05,2018/06,2018/07,2018/08,2018/09,2018/10,2018/11,2018/12,2019/01
0,AGUASCALIENTES,Minería,71.581928,74.538138,81.759683,72.966245,76.021630,67.741930,70.577510,75.160746,73.241484,71.103056,78.240948,67.380510,61.900116
1,AGUASCALIENTES,"Generación, transmisión y distribución de ener...",123.135766,125.902232,116.336919,131.971977,122.666208,134.363338,128.927324,124.895400,135.179778,133.347973,142.452951,126.612567,127.888741
2,AGUASCALIENTES,Construcción,122.091917,130.383091,126.659724,113.136473,115.405914,108.993267,116.819366,116.504343,117.430186,121.931559,126.966549,118.115246,115.432118
3,AGUASCALIENTES,Industrias manufactureras,156.279936,137.011376,143.705521,130.896569,143.841126,147.869162,123.645452,153.326693,148.927985,161.185942,158.353279,125.159600,127.898922
4,BAJA CALIFORNIA,Actividades secundarias,119.017000,118.761770,122.218695,120.953159,123.841961,124.877570,127.481205,131.474620,128.187831,133.690253,127.974122,121.101113,128.953189


In [14]:
merge_2 = pd.merge(PIB_df,catalogo_sector,on="Actividad_economica_agragada",how="left")

In [34]:
merge_2.head(1)

,Estado_calculado,Actividad_economica_agragada,2018/01,2018/02,2018/03,2018/04,2018/05,2018/06,2018/07,2018/08,2018/09,2018/10,2018/11,2018/12,2019/01,Sector
0,AGUASCALIENTES,Minería,71.581928,74.538138,81.759683,72.966245,76.02163,67.74193,70.57751,75.160746,73.241484,71.103056,78.240948,67.38051,61.900116,Mineria y energia


In [35]:
group_pib=pd.DataFrame(merge_2.groupby(["Estado_calculado","Sector"]).mean())

In [37]:
group_pib.head(1)

,,2018/01,2018/02,2018/03,2018/04,2018/05,2018/06,2018/07,2018/08,2018/09,2018/10,2018/11,2018/12,2019/01
Estado_calculado,Sector,,,,,,,,,,,,,
AGUASCALIENTES,Actividades secundarias,143.423253,133.763434,136.807314,124.356369,133.304994,133.934795,120.775552,139.804141,137.429717,146.718298,146.78649,122.079741,122.926209


In [43]:
group_pib=group_pib.reset_index(level=1,col_level=1)
group_pib=group_pib.reset_index(level=0,col_level=0)

In [45]:
group_pib.columns

Index(['Estado_calculado', 'Sector', '2018/01', '2018/02', '2018/03',
       '2018/04', '2018/05', '2018/06', '2018/07', '2018/08', '2018/09',
       '2018/10', '2018/11', '2018/12', '2019/01'],
      dtype='object')

In [46]:
group_cnbv=merge_1.groupby(["Estado_calculado","Sector"]).sum()
group_cnbv=group_cnbv.reset_index(level=1,col_level=1)
group_cnbv=group_cnbv.reset_index(level=0,col_level=0)

In [47]:
group_cnbv.columns

Index(['Estado_calculado', 'Sector', 'Cartera_Total', 'IMOR'], dtype='object')

In [51]:
new_df = pd.merge(group_cnbv,group_pib,  how='left', left_on=['Estado_calculado','Sector'], right_on = ['Estado_calculado','Sector'])

In [52]:
cnbv_vs_pib.head(25)

,Estado_calculado,Sector_x,Cartera_Total,IMOR,Sector_y,2018/01,2018/02,2018/03,2018/04,2018/05,2018/06,2018/07,2018/08,2018/09,2018/10,2018/11,2018/12,2019/01
0,AGUASCALIENTES,Actividades secundarias,2827.89,52.91,Actividades secundarias,143.423253,133.763434,136.807314,124.356369,133.304994,133.934795,120.775552,139.804141,137.429717,146.718298,146.786490,122.079741,122.926209
1,AGUASCALIENTES,Actividades secundarias,2827.89,52.91,Construccion,122.091917,130.383091,126.659724,113.136473,115.405914,108.993267,116.819366,116.504343,117.430186,121.931559,126.966549,118.115246,115.432118
2,AGUASCALIENTES,Actividades secundarias,2827.89,52.91,Industrias manufactureras,156.279936,137.011376,143.705521,130.896569,143.841126,147.869162,123.645452,153.326693,148.927985,161.185942,158.353279,125.159600,127.898922
3,AGUASCALIENTES,Actividades secundarias,2827.89,52.91,Mineria y energia,97.358847,100.220185,99.048301,102.469111,99.343919,101.052634,99.752417,100.028073,104.210631,102.225515,110.346949,96.996538,94.894428
4,AGUASCALIENTES,Construccion,5386.02,178.88,Actividades secundarias,143.423253,133.763434,136.807314,124.356369,133.304994,133.934795,120.775552,139.804141,137.429717,146.718298,146.786490,122.079741,122.926209
5,AGUASCALIENTES,Construccion,5386.02,178.88,Construccion,122.091917,130.383091,126.659724,113.136473,115.405914,108.993267,116.819366,116.504343,117.430186,121.931559,126.966549,118.115246,115.432118
6,AGUASCALIENTES,Construccion,5386.02,178.88,Industrias manufactureras,156.279936,137.011376,143.705521,130.896569,143.841126,147.869162,123.645452,153.326693,148.927985,161.185942,158.353279,125.159600,127.898922
7,AGUASCALIENTES,Construccion,5386.02,178.88,Mineria y energia,97.358847,100.220185,99.048301,102.469111,99.343919,101.052634,99.752417,100.028073,104.210631,102.225515,110.346949,96.996538,94.894428
8,AGUASCALIENTES,Industrias manufactureras,4250.33,41.99,Actividades secundarias,143.423253,133.763434,136.807314,124.356369,133.304994,133.934795,120.775552,139.804141,137.429717,146.718298,146.786490,122.079741,122.926209
9,AGUASCALIENTES,Industrias manufactureras,4250.33,41.99,Construccion,122.091917,130.383091,126.659724,113.136473,115.405914,108.993267,116.819366,116.504343,117.430186,121.931559,126.966549,118.115246,115.432118
